In [1]:
#conda install numpy scipy joblib scikit-learn --force-reinstall

In [2]:
#conda list

In [3]:
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from six import StringIO  
from IPython.display import Image  
import pydotplus
import re
import seaborn as sns
import time
from datetime import date

In [4]:
import pandas as pd 
import numpy as np 
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
from functools import reduce
import matplotlib as mpl
import math
import scipy
from scipy.spatial.distance import cdist
import random


In [5]:
today = date.today()
print(today.strftime("%d/%m/%Y"))
print(time.strftime("%H:%M:%S", time.localtime()))

30/06/2024
15:45:10


In [6]:
# 364
tissue=364
counts0_raw_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/cell_expression_0-0624.txt"
counts1_raw_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/cell_expression_1-0624.txt"
counts0_norm_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/cell_expression_normed_0-0624.txt"
counts1_norm_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/cell_expression_normed_1-0624.txt"
GeneNames0_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/GeneNames_0-0624.txt"
GeneNames1_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/GeneNames_1-0624.txt"
all_cells_IDs_0_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/CellsIDs_0-0624.txt"
all_cells_IDs_1_path = "/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/CellsIDs_1-0624.txt"


In [7]:
##### 0
#load Gene names and cell IDs into lists
GeneNames0 = pd.read_csv(GeneNames0_path, names=['colA'], header=None)
GeneNames0_list=GeneNames0['colA'].values.tolist()
GeneNames0_df = pd.DataFrame(GeneNames0_list,columns=['GeneName0'], index=GeneNames0)
print(np.shape(GeneNames0_list))

all_cells_IDs0 = pd.read_csv(all_cells_IDs_0_path, names = ['CellIDs0'],header=None)
all_cells_IDs_0_list=all_cells_IDs0['CellIDs0'].values.tolist()
print(np.shape(all_cells_IDs_0_list))

(298,)
(1178,)


In [8]:
##### 1
#load Gene names and cell IDs into lists
GeneNames1 = pd.read_csv(GeneNames1_path, names=['colA'], header=None)
GeneNames1_list=GeneNames1['colA'].values.tolist()
GeneNames1_df = pd.DataFrame(GeneNames1_list,columns=['GeneName1'], index=GeneNames1)
print(np.shape(GeneNames1_list))

all_cells_IDs1 = pd.read_csv(all_cells_IDs_1_path, names = ['CellIDs1'],header=None)
all_cells_IDs_1_list=all_cells_IDs1['CellIDs1'].values.tolist()
print(np.shape(all_cells_IDs_1_list))

(287,)
(447,)


In [9]:
####### 0
# load dataset into Pandas DataFrame
counts_raw0 = np.transpose(pd.read_csv(counts0_raw_path, sep='\t', header=None))
counts_norm0 = np.transpose(pd.read_csv(counts0_norm_path, sep='\t', header=None))
counts_raw0.columns = GeneNames0_list
counts_norm0.columns = GeneNames0_list

counts_raw0.index = pd.Series(all_cells_IDs_0_list).astype('string').radd('X'+str(tissue)+'.')
counts_norm0.index = pd.Series(all_cells_IDs_0_list).astype('string').radd('X'+str(tissue)+'.')
#counts_norm0.index = (pd.to_numeric(all_cells_IDs_0_list)*1000).round(0).astype(int)

In [10]:
####### 1
# load dataset into Pandas DataFrame
counts_raw1 = np.transpose(pd.read_csv(counts1_raw_path, sep='\t', header=None))
counts_norm1 = np.transpose(pd.read_csv(counts1_norm_path, sep='\t', header=None))
counts_raw1.columns = GeneNames1_list
counts_norm1.columns = GeneNames1_list
counts_raw1.index = pd.Series(all_cells_IDs_1_list).astype('string').radd('X'+str(tissue)+'.')
counts_norm1.index = pd.Series(all_cells_IDs_1_list).astype('string').radd('X'+str(tissue)+'.')

In [11]:
###### 0 
counts_raw0

,ACTA2,ACTG2,ACTR3B,ADGRL4,AGR2,AHR,AIF1,AKT1,ALB,ALDH1A3,...,TTC6,TTYH1,TYMS,UBE2C,UBE2T,VIM,XBP1,XCL1,ZEB1,ZNF571
X364.1.0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,1,8,0
X364.2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,1
X364.3.0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,2,4
X364.5.0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,1,0,0,2,1
X364.6.0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X364.24.074,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
X364.1.075,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.2.075,0,0,0,0,0,0,2,0,1,0,...,0,0,0,0,8,0,0,7,0,0
X364.7.075,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,3,0,0,1,0,0


In [12]:
###### 1
counts_raw1

,ACTA2,ACTG2,ACTR3B,ADGRL4,AHR,AIF1,AKT1,ALB,ALDH1A3,ANLN,...,TTC6,TTYH1,TYMS,UBE2C,UBE2T,VIM,XBP1,XCL1,ZEB1,ZNF571
X364.9.0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
X364.11.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.13.0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
X364.10.001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.12.001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X364.79.069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.4.071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.8.071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.2.074,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
######## remain cells which are presented in both matrices - they have nucleus transcripts and cytoplasmic transcripts
counts_raw0_N=counts_raw0.loc[list(counts_raw1.index)]
counts_raw0_N

,ACTA2,ACTG2,ACTR3B,ADGRL4,AGR2,AHR,AIF1,AKT1,ALB,ALDH1A3,...,TTC6,TTYH1,TYMS,UBE2C,UBE2T,VIM,XBP1,XCL1,ZEB1,ZNF571
X364.9.0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,3,0
X364.11.0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,5
X364.13.0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,3
X364.10.001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.12.001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X364.79.069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.4.071,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X364.8.071,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
X364.2.074,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#######
counts_raw0_N.to_csv('/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/counts_0_N_364.csv')

In [15]:
########
counts_raw1.to_csv('/home/beast/storage/Tal/from_beast2/Tal/RNA_velocity/6_tissues/364/counts_1_364.csv')